In [25]:
import sys
import torch
from torch import nn
import time
from tqdm import tqdm
import torch.optim as optim
import copy

# Specify where to find the data preparation class
sys.path.append('../../Data_Preparation')
from Preparation import CustomDataLoader

In [2]:
# InceptionV3 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
DIMENSIONS = 3
#SIZE = 256

In [3]:
# Instantiate the CustomDataLoader class for training
train_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=32, dataset_type="train", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader
test_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=32, dataset_type="test", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(train_data_loader):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([32, 3, 299, 299])
Labels Shape: torch.Size([32])
Labels: tensor([2, 6, 0, 3, 2])
Batch Index: 1
Inputs Shape: torch.Size([32, 3, 299, 299])
Labels Shape: torch.Size([32])
Labels: tensor([6, 3, 2, 5, 6])
Batch Index: 2
Inputs Shape: torch.Size([32, 3, 299, 299])
Labels Shape: torch.Size([32])
Labels: tensor([6, 5, 6, 0, 5])


In [45]:
# load up the InceptionV3 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_ft = models.inception_v3(pretrained=True)

model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(model_ft.fc.in_features, 10),
    nn.Linear(10, 7)
)

model = model.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=0.001)

In [ ]:
num_epochs = 25

for epoch in tqdm(range(num_epochs)):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    
    total_batch = len(train_data_loader.dataset)//32

    for i, (batch_images, batch_labels) in enumerate(train_data_loader):
        
        X = batch_images.to(device)
        Y = batch_labels.to(device)

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

model.eval()

correct = 0
total = 0

for images, labels in test_data_loader:
    
    images = images.to(device)
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.to(device)-1).sum()
    
print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 0/24
----------
Epoch [1/25], lter [5/876] Loss: 1.9459
Epoch [1/25], lter [10/876] Loss: 1.9082
Epoch [1/25], lter [15/876] Loss: 1.9744
Epoch [1/25], lter [20/876] Loss: 2.0286
Epoch [1/25], lter [25/876] Loss: 2.0308
Epoch [1/25], lter [30/876] Loss: 1.9939
Epoch [1/25], lter [35/876] Loss: 1.9843
Epoch [1/25], lter [40/876] Loss: 1.9548
Epoch [1/25], lter [45/876] Loss: 1.9837
Epoch [1/25], lter [50/876] Loss: 1.9853
Epoch [1/25], lter [55/876] Loss: 1.9757
Epoch [1/25], lter [60/876] Loss: 1.9699
Epoch [1/25], lter [65/876] Loss: 1.9789
Epoch [1/25], lter [70/876] Loss: 1.9876
Epoch [1/25], lter [75/876] Loss: 1.9915
Epoch [1/25], lter [80/876] Loss: 1.9935
Epoch [1/25], lter [85/876] Loss: 1.9770
Epoch [1/25], lter [90/876] Loss: 1.9529
Epoch [1/25], lter [95/876] Loss: 1.9444
Epoch [1/25], lter [100/876] Loss: 1.9724
Epoch [1/25], lter [105/876] Loss: 1.9823
Epoch [1/25], lter [110/876] Loss: 1.9896
Epoch [1/25], lter [115/876] Loss: 1.9575
Epoch [1/25], lter [120/876] Los

  4%|▍         | 1/25 [59:14<23:41:37, 3554.06s/it]

Epoch 1/24
----------
Epoch [2/25], lter [5/876] Loss: 1.9891
Epoch [2/25], lter [10/876] Loss: 1.9390
Epoch [2/25], lter [15/876] Loss: 1.9907
Epoch [2/25], lter [20/876] Loss: 1.9649
Epoch [2/25], lter [25/876] Loss: 1.9994
Epoch [2/25], lter [30/876] Loss: 1.9478
Epoch [2/25], lter [35/876] Loss: 1.9730
Epoch [2/25], lter [40/876] Loss: 1.9958
Epoch [2/25], lter [45/876] Loss: 2.0026
Epoch [2/25], lter [50/876] Loss: 1.9470
Epoch [2/25], lter [55/876] Loss: 1.9505
Epoch [2/25], lter [60/876] Loss: 1.9981
Epoch [2/25], lter [65/876] Loss: 1.9695
Epoch [2/25], lter [70/876] Loss: 1.9718
Epoch [2/25], lter [75/876] Loss: 1.9771
Epoch [2/25], lter [80/876] Loss: 2.0006
Epoch [2/25], lter [85/876] Loss: 1.9101
Epoch [2/25], lter [90/876] Loss: 1.9532
Epoch [2/25], lter [95/876] Loss: 1.8949
Epoch [2/25], lter [100/876] Loss: 1.9529
Epoch [2/25], lter [105/876] Loss: 1.9264
Epoch [2/25], lter [110/876] Loss: 1.9295
Epoch [2/25], lter [115/876] Loss: 1.9461
Epoch [2/25], lter [120/876] Los

# GARBAGE

In [ ]:
# loop over epochs
print("[INFO] training the network...")
startTime = time.time()
for e in tqdm(range(5)):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (i, (x, y)) in enumerate(train_data_loader):
		# send the input to the device
		#(x, y) = (x.to(config.DEVICE), y.to(config.DEVICE))
		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFunc(pred, y)
		# calculate the gradients
		loss.backward()
		# check if we are updating the model parameters and if so
		# update them, and zero out the previously accumulated gradients
		if (i + 2) % 2 == 0:
			opt.step()
			opt.zero_grad()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()

    	# switch off autograd
	with torch.no_grad():
		# set the model in evaluation mode
		model.eval()
		# loop over the validation set
		for (x, y) in test_data_loader:
			# send the input to the device
			#(x, y) = (x.to(config.DEVICE), y.to(config.DEVICE))
			# make the predictions and calculate the validation loss
			pred = model(x)
			totalValLoss += lossFunc(pred, y)
			# calculate the number of correct predictions
			valCorrect += (pred.argmax(1) == y).type(
				torch.float).sum().item()

    	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
	# calculate the training and validation accuracy
	trainCorrect = trainCorrect / len(trainDS)
	valCorrect = valCorrect / len(valDS)
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["train_acc"].append(trainCorrect)
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	H["val_acc"].append(valCorrect)
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, 5))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
		avgTrainLoss, trainCorrect))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}".format(
		avgValLoss, valCorrect))

# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))
# plot the training loss and accuracy
# plt.style.use("ggplot")
# plt.figure()
# plt.plot(H["train_loss"], label="train_loss")
# plt.plot(H["val_loss"], label="val_loss")
# plt.plot(H["train_acc"], label="train_acc")
# plt.plot(H["val_acc"], label="val_acc")
# plt.title("Training Loss and Accuracy on Dataset")
# plt.xlabel("Epoch #")
# plt.ylabel("Loss/Accuracy")
# plt.legend(loc="lower left")
# plt.savefig(config.WARMUP_PLOT)
# serialize the model to disk
torch.save(model, config.WARMUP_MODEL)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for inputs, labels in train_data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes
            epoch_acc = running_corrects.double() / dataset_sizes
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # deep copy the model
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [28]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


TypeError: max() received an invalid combination of arguments - got (InceptionOutputs, int), but expected one of:
 * (Tensor input, *, Tensor out)
 * (Tensor input, Tensor other, *, Tensor out)
 * (Tensor input, int dim, bool keepdim, *, tuple of Tensors out)
 * (Tensor input, name dim, bool keepdim, *, tuple of Tensors out)
